In [ ]:
!pip install -r requirements.txt 

In [2]:
!python cover_military2jsonl.py \
    --data_path data/military.json \
    --save_path data/military.jsonl

formatting..: 100%|███████████████████| 24739/24739 [00:00<00:00, 176714.53it/s]


In [ ]:
!python tokenize_dataset_rows.py \
    --jsonl_path data/military.jsonl \
    --save_path data/military \
    --max_seq_length 128

In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
sys.path.append("../")

In [2]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = AutoModel.from_pretrained("/dataQ/zhijian/LLMs/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dataQ/zhijian/anaconda3/envs/chatglm did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"zh-


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.59s/it]


In [3]:
tokenizer = AutoTokenizer.from_pretrained("/dataQ/zhijian/LLMs/chatglm-6b", trust_remote_code=True)

## Test before finetune

In [5]:
from cover2jsonl import format_example
import json


instructions = json.load(open("data/military.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/transformers/generation/utils.py:1445: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The dtype of attention mask (torch.int64) is not bool


Instruction: 战斗机涡轮前温度有什么用?
Answer: 战斗机涡轮前温度(涡轮进气道温度)是一个重要的飞行参数,通常被用于评估涡轮发动机的性能和可靠性。

涡轮发动机通过吸入空气来产生动力,其中一部分空气被冷却并在涡轮前通过。涡轮前温度是一个重要的指标,反映了涡轮发动机对冷却空气的处理能力。较高的涡轮前温度通常意味着涡轮发动机具有更强的处理能力,但同时也意味着需要更多的冷却空气来保持温度稳定。

在战斗机飞行中,涡轮前温度的测量和优化可以帮助发动机在高速飞行时保持稳定性和可靠性,从而提高战斗机的性能和飞行效率。因此,战斗机设计者通常会通过测量和优化涡轮前温度来优化发动机的性能和飞行特性。
### 1.Answer:
 原则上前涡轮温度升高后，燃烧更加充分，推力有所上升。但单纯增加涡轮前温会导致风扇寿命下降，WS10就出现过这种问题。 


Instruction: 上游二号是超音速反舰导弹吗
Answer: 上游二号是中国研制的一种超音速反舰导弹。虽然上游二号没有公开具体的飞行速度数据,但据报道,它采用高超音速飞行,具有较远的射程和较强的反舰能力。
### 2.Answer:
 不是，还是高亚音速的。上游2还是冥河的老底子，看看那个气动外形——卵型。就知道了，没法飞超音速。 


Instruction: 哪个国家激光武器最先进
Answer: 当今世界上最先进的激光武器国家之一是俄罗斯。俄罗斯在激光武器领域有着悠久的历史和深厚的积累,已经开发出了一系列先进的激光武器系统,如“瓦良格”号航母上使用的激光炮、苏-35战斗机使用的激光器等。

美国也是激光武器领域的领先国家之一,其军队已经装备了一系列先进的激光武器系统,如“激光器家族”、激光照射器等。英国、法国、中国、日本等其他国家也在不同程度上进行了激光武器的研发和应用。

需要注意的是,激光武器是一种高度技术化的兵器,其性能往往受到多种因素的影响,如激光器的功率、准确度、稳定性等,因此不同国家在激光武器
### 3.Answer:
 激光武器是一种利用定向发射的激光束直接毁伤目标或使之失效的定向能武器。武器系统主要由激光器和跟踪、瞄准、发射装置等部分组成。目前通常采用的激光器有化学激光器、固体激光器、二氧化碳激光器等。激光武器具有攻击速度快、转向灵活、可实现精确打击、不受电磁干扰等优点，但也存在昴受天

In [11]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [12]:
import datasets

dataset_path = "data/military/"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

In [13]:
from transformers import Trainer, HfArgumentParser


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss

In [15]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

Step,Training Loss
50,2.091900
100,1.787700
150,2.020000
200,1.673600
250,1.978900
300,1.534500
350,1.611100
400,1.620300
450,1.778000
500,1.610000


/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1500, training_loss=1.4622032979329427, metrics={'train_runtime': 474.9934, 'train_samples_per_second': 3.158, 'train_steps_per_second': 3.158, 'total_flos': 3781851053211648.0, 'train_loss': 1.4622032979329427, 'epoch': 3.0})

## Test After finetune

In [6]:
from cover2jsonl import format_example
import json


instructions = json.load(open("data/military.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2636336/1101185756.py:5 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2636336/1101185756.py'                      │
│                                                                                                  │
│ /dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/json/__init__.py:293 in load                │
│                                                                                                  │
│   290 │   To use a custom ``JSONDecoder`` subclass, specify it with the ``cls``                  │
│   291 │   kwarg; otherwise ``JSONDecoder`` is used.                                              │
│   292 │   """                                                                                    │
│ ❱ 293 │   return loads(fp.read(),                                                                │
│   294 │   │   cls=cls, object_hook=object_hook,                                                  │
│   295 │   │   parse_float=parse_float, parse_int=parse_int,                                      │
│   296 │   │   parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)          │
│                                                                                                  │
│ /dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/encodings/ascii.py:26 in decode             │
│                                                                                                  │
│   23                                                                                             │
│   24 class IncrementalDecoder(codecs.IncrementalDecoder):                                        │
│   25 │   def decode(self, input, final=False):                                                   │
│ ❱ 26 │   │   return codecs.ascii_decode(input, self.errors)[0]                                   │
│   27                                                                                             │
│   28 class StreamWriter(Codec,codecs.StreamWriter):                                              │
│   29 │   pass                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 62: ordinal not in range(128)

In [7]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))